In [2]:
import sys
sys.path.append('..')

In [3]:
from utils import io

In [4]:
import numpy as np
import os.path as osp

In [39]:
data_root = '/home/sxr/data/social_and_user_item/datasets/instance_pokec'

In [40]:
indptr = io.load_pickle(osp.join(data_root, 'train_csr_indptr.pkl'))
degrees = indptr[1:] - indptr[:-1]

In [41]:
test_set = io.load_pickle(osp.join(data_root, 'test.pkl'))
src_list = test_set['src']
pos_list = test_set['pos_list']

In [42]:
src_list

array([     1, 131078,     10, ..., 393199, 393201, 262133], dtype=int32)

In [43]:
src_degrees = degrees[src_list]

In [44]:
p = []
src_list_dict = {}
pos_list_dict = {}
degree_info = []
for k in range(1, 11):
    pk = np.percentile(src_degrees, k*10)
    degree_info.append((k*10, pk))
    p.append(pk)
    print(k*10, pk)
    
    key = 'p' + str(k*10)
    src_list_dict[key] = []
    pos_list_dict[key] = []
degree_info = np.array(degree_info)
np.savetxt(osp.join(data_root, 'test_set-src-outdegree-percentile.txt'), degree_info, fmt='%.1f')

10 7.0
20 10.0
30 14.0
40 18.0
50 23.0
60 30.0
70 38.0
80 51.0
90 76.0
100 8309.0


In [45]:
len(src_list)

48660

In [46]:
cnt = 0
for s, pos in zip(src_list, pos_list):
    d = degrees[s]
    for k in range(1, 11):
        if d <= p[k - 1]:
            key = 'p' + str(k*10)
            src_list_dict[key].append(s)
            pos_list_dict[key].append(pos)
            cnt += 1
            break
assert cnt == len(src_list)

In [47]:
for key in src_list_dict:
    src_list_dict[key] = np.array(src_list_dict[key])

In [48]:
src_list_dict

{'p10': array([ 393233,  524349,  655443, ..., 1048509,  786373, 1048544],
       dtype=int32),
 'p20': array([ 131078, 1048643,  786560, ...,  261990,  917446, 1441744],
       dtype=int32),
 'p30': array([ 917533, 1048609,  262213, ...,  786345,  786352,  524247],
       dtype=int32),
 'p40': array([262276, 524464, 524490, ..., 524204, 524221, 917486], dtype=int32),
 'p50': array([ 524341, 1048741,     166, ...,  786329,  655285,  131008],
       dtype=int32),
 'p60': array([ 131088,  786477,  655424, ...,  917323, 1048511,  917478],
       dtype=int32),
 'p70': array([262186, 524337, 655418, ..., 130962, 393199, 393201], dtype=int32),
 'p80': array([    10, 393243,     62, ..., 917420, 786351, 262107], dtype=int32),
 'p90': array([524314, 262173,     65, ..., 393103, 393178, 655324], dtype=int32),
 'p100': array([     1, 131083, 262204, ..., 130970, 262045, 262133], dtype=int32)}

In [49]:
for key in src_list_dict:
    io.save_pickle(osp.join(data_root, 'test-' + key + '.pkl'), 
        {
            'src': src_list_dict[key],
            'pos_list': pos_list_dict[key]
        }
    )